# Getting Corpus 
Searching google for the references and getting articles corresponding each reference

Making a change to the reference parsing algorithm : The current __get_hotwords__ method does not include numbers so an external method is created which is able to add numbers as they are relevant contextual information 
<br/>
        
    case in point, the following sentence 
    - About 1500 people were evacuated from dangerous areas of Luhansk region
    is translated as 
    - people dangerous areas luhansk region 
    
## Methodolgy

Since there are ~3000 references, we have decided to split the article dataset into 
l; oj;neipmvsd[lf 9w8uehf jwd][f
    

In [2]:
import pandas as pd
from textblob import TextBlob
from newspaper import Article
from datetime import timedelta
import spacy
from collections import Counter
from string import punctuation

from googlesearch import search


import sys
sys.path.append('../')
import utility_functions


In [3]:
nlp = spacy.load("en_core_web_md")

In [4]:
reference = pd.read_csv('../data/processed/input-data.csv',parse_dates=['time_parsed'])

In [5]:
len(reference)

2921

## Testing on one reference 

In [6]:
current_ref = reference.iloc[203]

In [7]:
current_ref

Unnamed: 0                                                      0
Unnamed: 0.1                                                    0
uid                          84c851c6-29c7-4c56-a1c5-a87217aace82
title           About 1500 people were evacuated from dangerou...
time                                                2 minutes ago
source                          https://t.me/ukrinform_news/62603
time-capture                           2022-04-03 10:42:11.311383
sc                                                          False
domain          data:image/svg+xml;base64,PHN2ZyB2ZXJzaW9uPSIx...
full-text       SourceOn live map\r\nTell friends\r\n2 minutes...
sc2                                                         False
geo                                               48°52′N 38°24′E
time_parsed                                   2022-04-03 10:40:21
Name: 203, dtype: object

In [8]:
title = current_ref['title']
titleHotwords = utility_functions.get_hotwords(title, nlp)
modifiedHotwords = utility_functions.add_numerics(title,titleHotwords)
timeFrom = str(current_ref['time_parsed'].date()) # Date at the time of reporting
timeTo = str((current_ref['time_parsed'] + timedelta(days=2)).date())  # Date 2 days from the date of reporting 

Number 1500 not found in hotwords, adding


In [9]:
SEARCHTERM = ' '.join(modifiedHotwords) + ' after:' +timeFrom + ' before:' + timeTo
print(f'Searching for\n----- {SEARCHTERM}')

Searching for
----- people dangerous areas luhansk region 1500 after:2022-04-03 before:2022-04-05


In [10]:
# searches = 0 
# for searchResult in search(SEARCHTERM):
#     if 'liveuamap.com' in searchResult or 'wikipedia.org' in searchResult :
#         continue
        
#     searches = searches + 1 
#     print(searchResult)
    
    
#     article_parse = Article(searchResult)
#     article_parse.download()
#     article_parse.parse()
#     article_author = article_parse.authors
#     article_date = article_parse.publish_date
#     article_text = article_parse.text

#     if searches >10:
#         break

## Applying on all dataset

In [11]:
current_ref

Unnamed: 0                                                      0
Unnamed: 0.1                                                    0
uid                          84c851c6-29c7-4c56-a1c5-a87217aace82
title           About 1500 people were evacuated from dangerou...
time                                                2 minutes ago
source                          https://t.me/ukrinform_news/62603
time-capture                           2022-04-03 10:42:11.311383
sc                                                          False
domain          data:image/svg+xml;base64,PHN2ZyB2ZXJzaW9uPSIx...
full-text       SourceOn live map\r\nTell friends\r\n2 minutes...
sc2                                                         False
geo                                               48°52′N 38°24′E
time_parsed                                   2022-04-03 10:40:21
Name: 203, dtype: object

### Test Batch

In [12]:
# Batch 1 
start_1 = 0 
end_1 = 2
# end_1 = int(len(reference))

print(f'Getting Dataset From {start_1} to {end_1} ')

batch_1 = []

for index in range(start_1,end_1):
    current_ref = reference.iloc[index]
    title = current_ref['title']
    titleHotwords = utility_functions.get_hotwords(title, nlp)
    uid = current_ref['uid']
    modifiedHotwords = utility_functions.add_numerics(title,titleHotwords)
    timeFrom = str(current_ref['time_parsed'].date()) # Date at the time of reporting
    timeTo = str((current_ref['time_parsed'] + timedelta(days=2)).date())  # Date 2 days from the date of reporting 
    
    SEARCHTERM = ' '.join(modifiedHotwords) + ' after:' +timeFrom + ' before:' + timeTo
    print(f'Searching for\n----- {SEARCHTERM}')
    
    searches = 0 
    for searchResult in search(SEARCHTERM):
        if 'liveuamap.com' in searchResult or 'wikipedia.org' in searchResult :
            continue

        
        print(searchResult,end = ' ---')
        
        try:

            article_parse = Article(searchResult)
            article_parse.download()
            article_parse.parse()
            article_author = article_parse.authors
            article_date = article_parse.publish_date
            article_text = article_parse.text

            batch_1.append({
                'uid' : uid,
                'article_author' : article_author,
                'article_date' : article_date,
                'article_text' : article_text,
                'article_url' : searchResult
            })
            searches = searches + 1 
            printm ('Done')
        except Exception:
            print('Failed:Skipping')
        if searches >10:
            break

    

Getting Dataset From 0 to 2 
Searching for
----- czech prime minister russian army war crimes ukraine after:2022-04-03 before:2022-04-05
https://www.boston.com/news/politics/2022/04/03/live-updates-zelenskyy-russian-attack-is-genocide/ ---Done
https://www.tampabay.com/news/nation-world/2022/04/03/live-updates-ukraine-410-civilian-bodies-found-near-kyiv/ ---Done
https://www.itv.com/news/2022-04-03/ukraine-describes-scenes-of-genocide-in-liberated-towns-near-kyiv ---Done
https://www.nytimes.com/live/2022/03/03/world/russia-ukraine ---Done
https://www.usatoday.com/story/news/politics/2022/03/04/ukraine-russia-invasion-live-updates/9369419002/ ---Done
https://news.rthk.hk/rthk/en/component/k2/1642338-20220404.htm ---Done
https://fortune.com/2022/04/03/uk-prime-minister-boris-johnson-calls-attacks-in-ukraine-despicable-russia-putin-war-crimes/ ---Done
https://www.wionews.com/world/live-russia-claims-to-have-besieged-two-big-cities-in-ukraine-455211 ---Done
https://www.post-gazette.com/news/

,uid,article_author,article_date,article_text,article_url
0,405ca5ff-f11a-42bf-8ba7-4856c9c88921,"[The Associated Press, Associated Press]",2022-04-03 00:00:00,Politics Live updates | Ukraine’s leader asks ...,https://www.boston.com/news/politics/2022/04/0...
1,405ca5ff-f11a-42bf-8ba7-4856c9c88921,[Associated Press],2022-04-03 00:00:00,"KYIV, Ukraine — Ukraine’s prosecutor-general s...",https://www.tampabay.com/news/nation-world/202...
2,405ca5ff-f11a-42bf-8ba7-4856c9c88921,[],2022-04-03 00:00:00,This video contains distressing images Play Br...,https://www.itv.com/news/2022-04-03/ukraine-de...
3,405ca5ff-f11a-42bf-8ba7-4856c9c88921,"[The New York Times, Marc Santora, Anton Troia...",2022-03-03 00:00:00,A fire broke out early Friday at a complex in ...,https://www.nytimes.com/live/2022/03/03/world/...
4,405ca5ff-f11a-42bf-8ba7-4856c9c88921,[],2022-03-04 00:00:00,Editor's note: This page recaps the news from ...,https://www.usatoday.com/story/news/politics/2...
5,405ca5ff-f11a-42bf-8ba7-4856c9c88921,[],2022-04-04 00:00:00,Russia denies killing civilians in Bucha\n\nRu...,https://news.rthk.hk/rthk/en/component/k2/1642...
6,405ca5ff-f11a-42bf-8ba7-4856c9c88921,[],2022-04-03 00:00:00,This is not the 2008 housing bubble all over a...,https://fortune.com/2022/04/03/uk-prime-minist...
7,405ca5ff-f11a-42bf-8ba7-4856c9c88921,[],None,Switzerland adopts new EU sanctions on Russia\...,https://www.wionews.com/world/live-russia-clai...
8,405ca5ff-f11a-42bf-8ba7-4856c9c88921,[],2022-04-03 00:00:00,,https://www.post-gazette.com/news/world/2022/0...
9,405ca5ff-f11a-42bf-8ba7-4856c9c88921,[Rob Schmitz Is Npr'S International Correspond...,2022-04-03 00:00:00,"MILES PARKS, HOST:\n\nIn Hungary, it's electio...",https://www.wunc.org/2022-04-03/hungarys-orban...


### Final Batches

In [17]:
# Batch n 

BATCH_NO = 0 
SCRAPED_BY = 'sam'

startIndex = BATCH_NO * 100
endIndex = start_1 + 100

print(f'Getting Dataset From {startIndex} to {endIndex} ')

batch = []

for index in range(startIndex,endIndex):
    current_ref = reference.iloc[index]
    title = current_ref['title']
    titleHotwords = utility_functions.get_hotwords(title, nlp)
    uid = current_ref['uid']
    modifiedHotwords = utility_functions.add_numerics(title,titleHotwords)
    timeFrom = str(current_ref['time_parsed'].date()) # Date at the time of reporting
    timeTo = str((current_ref['time_parsed'] + timedelta(days=2)).date())  # Date 2 days from the date of reporting 
    
    SEARCHTERM = ' '.join(modifiedHotwords) + ' after:' +timeFrom + ' before:' + timeTo
    print(f'Searching for\n----- {SEARCHTERM}')
    
    searches = 0 
    for searchResult in search(SEARCHTERM):
        if 'liveuamap.com' in searchResult or 'wikipedia.org' in searchResult :
            continue

        
        print(searchResult,end = ' ---')
        
        try:

            article_parse = Article(searchResult)
            article_parse.download()
            article_parse.parse()
            article_author = article_parse.authors
            article_date = article_parse.publish_date
            article_text = article_parse.text

            batch.append({
                'uid' : uid,
                'article_author' : article_author,
                'article_date' : article_date,
                'article_text' : article_text,
                'article_url' : searchResult
            })
            searches = searches + 1 
            print('Done')
        except Exception:
            print('Failed:Skipping')
        if searches >10:
            break

            
            
    

Getting Dataset From 0 to 2 
Searching for
----- czech prime minister russian army war crimes ukraine after:2022-04-03 before:2022-04-05
https://www.boston.com/news/politics/2022/04/03/live-updates-zelenskyy-russian-attack-is-genocide/ ---Done
https://www.tampabay.com/news/nation-world/2022/04/03/live-updates-ukraine-410-civilian-bodies-found-near-kyiv/ ---Done
https://www.itv.com/news/2022-04-03/ukraine-describes-scenes-of-genocide-in-liberated-towns-near-kyiv ---Done
https://www.nytimes.com/live/2022/03/03/world/russia-ukraine ---Done
https://www.usatoday.com/story/news/politics/2022/03/04/ukraine-russia-invasion-live-updates/9369419002/ ---Done
https://news.rthk.hk/rthk/en/component/k2/1642338-20220404.htm ---Done
https://fortune.com/2022/04/03/uk-prime-minister-boris-johnson-calls-attacks-in-ukraine-despicable-russia-putin-war-crimes/ ---Done
https://www.wionews.com/world/live-russia-claims-to-have-besieged-two-big-cities-in-ukraine-455211 ---Done
https://www.post-gazette.com/news/

In [18]:
pd.DataFrame(batch).to_csv(f'../data/processed/{str(BATCH_NO)}_{SCRAPED_BY}_{startIndex}to{endIndex}.csv')